In [ ]:
### Whales Detector (:

In [ ]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import time

# API endpoint e chiave
base_url = "https://fapi.binance.com"
endpoint = "/fapi/v1/historicalTrades"
api_key = "Your Apy Key"
symbol = "BTCUSDT"
limit = 1000

# Funzione per convertire timestamp in formato datetime
def convert_to_datetime(timestamp):
    return datetime.fromtimestamp(timestamp / 1000.0)

# Funzione per creare il database e la tabella
def create_db():
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS trades
                 (id INTEGER PRIMARY KEY, 
                  timestamp TEXT, 
                  trade_type TEXT, 
                  price REAL, 
                  qty REAL)''')
    conn.commit()
    conn.close()

# Funzione per ottenere l'ultimo ID dal database
def get_last_trade_id():
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    c.execute('SELECT MAX(id) FROM trades')
    last_id = c.fetchone()[0]
    conn.close()
    return last_id

# Funzione per salvare i trade nel database
def save_trades_to_db(trades):
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    for trade in trades:
        c.execute('''INSERT OR IGNORE INTO trades (id, timestamp, trade_type, price, qty)
                     VALUES (?, ?, ?, ?, ?)''', 
                  (trade['id'], convert_to_datetime(trade['time']), 
                   'bid' if trade['isBuyerMaker'] else 'ask', 
                   trade['price'], trade['qty']))
    conn.commit()
    conn.close()

# Funzione per caricare i dati dal database
def load_trades_from_db():
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    c.execute('SELECT * FROM trades WHERE qty > 10')  # Filtra per quantità sopra i 2 BTC
    rows = c.fetchall()
    conn.close()
    
    # Converti i dati in DataFrame
    df = pd.DataFrame(rows, columns=['id', 'Timestamp', 'Trade Type', 'Price', 'Qty'])
    return df

# Funzione per ottenere i dati dall'API Binance utilizzando l'ultimo ID
def get_historical_trades(fromId=None):
    url = base_url + endpoint
    headers = {"X-MBX-APIKEY": api_key}
    params = {"symbol": symbol, "limit": limit}
    
    if fromId:
        params['fromId'] = fromId
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # Solleva un'eccezione in caso di errore nella richiesta
    
    return response.json()

# Funzione per aggiornare i dati (online) e recuperare trade mancanti
def update_data():
    try:
        last_trade_id = get_last_trade_id()  # Ottieni l'ID dell'ultimo trade salvato
        if last_trade_id is None:
            last_trade_id = 1  # Se non ci sono trade, inizia dall'ID 1
        
        # Continua a recuperare blocchi di trade finché ci sono nuovi trade disponibili
        while True:
            trades = get_historical_trades(fromId=last_trade_id + 1)
            if not trades:
                break  # Se non ci sono più trade, esci dal ciclo
            
            save_trades_to_db(trades)
            last_trade_id = trades[-1]['id']  # Aggiorna l'ultimo ID con l'ultimo trade ottenuto
        
        messagebox.showinfo("Successo", "Dati aggiornati con successo!")
        display_data()
    except requests.exceptions.RequestException as e:
        messagebox.showwarning("Errore", f"Errore durante il recupero dei dati: {e}")
    except Exception as e:
        messagebox.showwarning("Errore", f"Si è verificato un errore: {e}")

# Funzione per verificare la connessione
def check_internet_connection():
    try:
        requests.get('https://www.google.com', timeout=3)
        return True
    except requests.ConnectionError:
        return False

# Funzione per visualizzare i dati nella GUI con colorazione delle righe
def display_data():
    df = load_trades_from_db()
    
    # Clear the old data in the treeview
    for row in tree.get_children():
        tree.delete(row)
    
    # Add new rows to the treeview with colors based on trade type
    for index, row in df.iterrows():
        color = "green" if row['Trade Type'] == 'bid' else "red"  # Verde per 'bid', rosso per 'ask'
        tree.insert("", "end", values=(row['id'], row['Timestamp'], row['Trade Type'], row['Price'], row['Qty']),
                    tags=(color,))
    
    # Configura gli stili per le righe
    tree.tag_configure('green', background='lightgreen')  # Verde chiaro per bid
    tree.tag_configure('red', background='lightcoral')    # Rosso chiaro per ask

# Funzione per l'aggiornamento automatico
def auto_update():
    if check_internet_connection():
        update_data()
    else:
        messagebox.showwarning("Connessione assente", "Non è possibile aggiornare i dati. Sei offline.")
    root.after(60000, auto_update)  # Ripeti ogni 60 secondi

# Creazione della finestra principale con Tkinter
root = tk.Tk()
root.title("Whales Monitor")
root.geometry("500x800")

# Creazione di un frame per contenere il Treeview e la Scrollbar
frame = tk.Frame(root)
frame.pack(fill='both', expand=True)

# Creazione della scrollbar verticale
scrollbar = tk.Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Creazione della tabella (Treeview)
tree = ttk.Treeview(frame, columns=('ID', 'Timestamp', 'Trade Type', 'Price', 'Qty'), show='headings', yscrollcommand=scrollbar.set)

# Configura la scrollbar per controllare il Treeview
scrollbar.config(command=tree.yview)

# Definisci le intestazioni della tabella
tree.heading('ID', text='Trade ID')
tree.heading('Timestamp', text='Timestamp')
tree.heading('Trade Type', text='Trade Type')
tree.heading('Price', text='Price')
tree.heading('Qty', text='Qty')

tree.pack(fill='both', expand=True)

# Bottone per aggiornare manualmente i dati
refresh_button = tk.Button(root, text="Aggiorna Dati", command=update_data)
refresh_button.pack()

# Avvia il caricamento iniziale dei dati e l'aggiornamento automatico
create_db()
display_data()
auto_update()

# Avvia la finestra
root.mainloop()

In [ ]:
##Whales Detector 2.0 (:

In [ ]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import threading

# API endpoint e chiave
base_url = "https://fapi.binance.com"
endpoint = "/fapi/v1/historicalTrades"
api_key = "Your Api Key"
symbol = "BTCUSDT"
limit = 1000

# Variabile globale per la quantità minima
min_qty = 50 # Quantità minima predefinita

# Funzione per convertire timestamp in formato datetime
def convert_to_datetime(timestamp):
    return datetime.fromtimestamp(timestamp / 1000.0)

# Funzione per creare il database e la tabella
def create_db():
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS trades
                 (id INTEGER PRIMARY KEY, 
                  timestamp DATETIME, 
                  trade_type TEXT, 
                  price REAL, 
                  qty REAL)''')
    conn.commit()
    conn.close()

# Funzione per ottenere l'ultimo Timestamp dal database
def get_last_trade_id():
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    c.execute('SELECT MAX(timestamp) FROM trades')
    last_id = c.fetchone()[0]
    conn.close()
    return last_id

# Funzione per salvare i trade nel database
def save_trades_to_db(trades):
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()
    for trade in trades:
        c.execute('''INSERT OR IGNORE INTO trades (id, timestamp, trade_type, price, qty)
                     VALUES (?, ?, ?, ?, ?)''', 
                  (trade['id'], convert_to_datetime(trade['time']), 
                   'bid' if trade['isBuyerMaker'] else 'ask',  # 'bid' e 'ask' in minuscolo
                   trade['price'], trade['qty']))
    conn.commit()
    conn.close()

# Funzione per caricare i dati dal database in base alla quantità e all'intervallo temporale filtrati
def load_trades_from_db(min_qty, start_time=None, end_time=None, limit=100):
    conn = sqlite3.connect('btc_trades.db')
    c = conn.cursor()

    # Aggiungi filtro temporale
    query = 'SELECT * FROM trades WHERE qty > ?'
    params = [min_qty]
    if start_time and end_time:
        query += ' AND timestamp BETWEEN ? AND ?'
        params.append(start_time)
        params.append(end_time)

    query += ' ORDER BY id DESC LIMIT ?'
    params.append(limit)

    c.execute(query, tuple(params))
    rows = c.fetchall()
    conn.close()
    
    # Converti i dati in DataFrame
    df = pd.DataFrame(rows, columns=['id', 'Timestamp', 'Trade Type', 'Price', 'Qty'])
    return df

# Funzione per ottenere i dati dall'API Binance utilizzando l'ultimo ID e gestire i blocchi di 1000 trade
def get_historical_trades(fromId=None):
    url = base_url + endpoint
    headers = {"X-MBX-APIKEY": api_key}
    params = {"symbol": symbol, "limit": 1000}  # Binance consente un massimo di 1000 trade per richiesta
    
    if fromId:
        params['fromId'] = fromId  # Se abbiamo un fromId, lo passiamo come parametro
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # Restituisci i dati JSON
    else:
        response.raise_for_status()  # Solleva un'eccezione in caso di errore


# Funzione per aggiornare i dati (online) e recuperare trade mancanti a blocchi di 1000
def update_data():
    try:
        last_id = get_last_trade_id()  # Ottieni l'ID dell'ultimo trade salvato
        if last_id is None:
            last_id = 1  # Se non ci sono trade nel database, inizia dall'ID 1
        
        # Continua a recuperare blocchi di trade finché ci sono nuovi trade disponibili
        while True:
            trades = get_historical_trades(fromId=last_id + 1)
            
            if not trades:
                break  # Se non ci sono più trade, esci dal ciclo
            
            save_trades_to_db(trades)  # Salva i trade nel database
            
            last_id = trades[-1]['id']  # Aggiorna l'ultimo ID con l'ultimo trade ottenuto

        messagebox.showinfo("Successo", "Dati aggiornati con successo!")
        display_data()  # Mostra i dati aggiornati
    except requests.exceptions.RequestException as e:
        messagebox

# Funzione per visualizzare i dati nella GUI con colorazione delle righe
def display_data(limit=100):
    global min_qty  # Usa la variabile globale per il filtro
    # Ottieni i valori temporali dagli input utente
    start_time = start_time_entry.get()
    end_time = end_time_entry.get()

    # Filtra solo se entrambi gli intervalli temporali sono forniti
    if start_time and end_time:
        try:
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
            end_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            messagebox.showwarning("Errore", "Formato data non valido. Usa 'YYYY-MM-DD HH:MM:SS'.")
            return
    else:
        start_time, end_time = None, None

    df = load_trades_from_db(min_qty, start_time, end_time, limit)

    # Clear the old data in the treeview
    for row in tree.get_children():
        tree.delete(row)
    
    # Aggiungi nuove righe al Treeview con colori basati sul tipo di trade
    for index, row in df.iterrows():
        color = "green" if row['Trade Type'] == 'bid' else "red"  # Verde per 'bid', rosso per 'ask'
        price = '{:.2f}'.format(float(row['Price']))  # Formatta il prezzo
        tree.insert("", "end", values=(row['id'], row['Timestamp'], row['Trade Type'], price, row['Qty']),
                    tags=(color,))
    
    # Configura gli stili per le righe
    tree.tag_configure('green', background='lightgreen')  # Verde chiaro per bid
    tree.tag_configure('red', background='lightcoral')    # Rosso chiaro per ask

# Funzione per aggiornare la quantità minima inserita dall'utente
def update_min_qty():
    global min_qty
    try:
        # Recupera il valore inserito dall'utente
        min_qty = float(min_qty_entry.get())
        display_data()  # Aggiorna la visualizzazione dei dati
        messagebox.showinfo("Filtro aggiornato", f"Il filtro è stato aggiornato: Min qty = {min_qty} BTC")
    except ValueError:
        messagebox.showwarning("Errore", "Per favore, inserisci un valore numerico valido per la quantità minima.")

# Funzione per aggiornare automaticamente in background
def auto_update():
    threading.Thread(target=update_data).start()

# Creazione della finestra principale con Tkinter
root = tk.Tk()
root.title("Whales Monitor")
root.geometry("500x800")

# Creazione di un frame per contenere il Treeview e la Scrollbar
frame = tk.Frame(root)
frame.pack(fill='both', expand=True)

# Creazione della scrollbar verticale
scrollbar = tk.Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Creazione della tabella (Treeview)
tree = ttk.Treeview(frame, columns=('ID', 'Timestamp', 'Trade Type', 'Price', 'Qty'), show='headings', yscrollcommand=scrollbar.set)

# Configura la scrollbar per controllare il Treeview
scrollbar.config(command=tree.yview)

# Definisci le intestazioni della tabella
tree.heading('ID', text='Trade ID')
tree.heading('Timestamp', text='Timestamp')
tree.heading('Trade Type', text='Trade Type')
tree.heading('Price', text='Price')
tree.heading('Qty', text='Qty')

tree.pack(fill='both', expand=True)

# Frame per i filtri
filter_frame = tk.Frame(root)
filter_frame.pack(pady=10)

# Entry per inserire la quantità minima
min_qty_label = tk.Label(filter_frame, text="Min Qty:")
min_qty_label.grid(row=0, column=0)

min_qty_entry = tk.Entry(filter_frame, width=10)
min_qty_entry.grid(row=0, column=1)

# Entry per inserire l'intervallo di tempo (start time e end time)
start_time_label = tk.Label(filter_frame, text="Inizio:")
start_time_label.grid(row=1, column=0)

start_time_entry = tk.Entry(filter_frame, width=20)
start_time_entry.grid(row=1, column=1)

end_time_label = tk.Label(filter_frame, text="Fine:")
end_time_label.grid(row=2, column=0)

end_time_entry = tk.Entry(filter_frame, width=20)
end_time_entry.grid(row=2, column=1)

# Pulsante per aggiornare la quantità minima
update_button = tk.Button(filter_frame, text="Aggiorna Min Qty", command=update_min_qty)
update_button.grid(row=3, columnspan=2, pady=5)

# Pulsante per aggiornare i dati
update_data_button = tk.Button(root, text="Aggiorna Dati", command=auto_update)
update_data_button.pack(pady=5)

# Avvia il programma
create_db()
display_data()

root.mainloop() 